In [1]:
import pandas as pd
import tushare as ts
import numpy as np
import matplotlib.pyplot as plt
"""
Read data
"""
input_path = '../data/三家银行A H分钟收盘价.xlsx'
df_input = pd.read_excel(input_path,converters={'600036.SH':str, '3968.HK':str})

In [2]:
"""
Seperate date and time
"""
df_AH= pd.merge(df_input[['600036.SH', '招商银行','工商银行','农业银行']].drop([0,1]).dropna(), \
    df_input[['3968.HK','招商银行.1','工商银行.1','农业银行.1']].drop([0,1]).dropna(), \
        how='inner',left_on='600036.SH', right_on='3968.HK').drop('3968.HK',axis=1).rename(columns= {'600036.SH':'date'})
df_date = df_AH['date'].str.split(' ',expand=True).rename(columns={0:'date', 1:'time'})
df_AH['time'] = df_AH['date'].copy()
df_AH['date'] = df_date['date'].str.replace('-','')
df_AH[['招商银行','工商银行','农业银行','招商银行.1','工商银行.1','农业银行.1']] = df_AH[['招商银行','工商银行','农业银行','招商银行.1','工商银行.1','农业银行.1']].astype(float)

"""
Exchange rate
    -convert to USD
    -use daily exchange to match stock price
"""
pro = ts.pro_api('00d803b166f55fc30c178d74c158985136010d6bd19271b182059eef')
start_date = '2021-01-04'
end_date = '2022-02-25'
df_CNY = pro.fx_daily(ts_code='USDCNH.FXCM', start_date=start_date, end_date=end_date).iloc[::-1].reset_index(drop=True)
df_HKD = pro.fx_daily(ts_code='USDHKD.FXCM', start_date=start_date, end_date=end_date).iloc[::-1].reset_index(drop=True)
ER_feature_list = ['trade_date', 'ask_close']
# Exchage Rate
df_ER = df_CNY[ER_feature_list].rename(columns={'ask_close':'CNY2USD'}).merge(df_HKD[ER_feature_list].rename(columns={'ask_close':'HKD2USD'}))
df_AH = pd.merge(df_AH, df_ER, left_on='date', right_on='trade_date', how='left').dropna().drop('trade_date',axis=1)
"""
Uniform the measuring unit to USD
"""
df_AH['招商银行'] = df_AH['招商银行'] / df_AH['CNY2USD']
df_AH['招商银行.1'] = df_AH['招商银行.1'] / df_AH['HKD2USD']
df_AH['工商银行'] = df_AH['工商银行'] / df_AH['CNY2USD']
df_AH['工商银行.1'] = df_AH['工商银行.1'] / df_AH['HKD2USD']
df_AH['农业银行'] = df_AH['农业银行'] / df_AH['CNY2USD']
df_AH['农业银行.1'] = df_AH['农业银行.1'] / df_AH['HKD2USD']

In [3]:
"""
Save the preprocessed data
"""
df_AH.to_excel('../data/df_AH.xlsx')